In [1]:
"""
https://watlab-blog.com/2022/10/08/2d-diffusion/
https://watlab-blog.com/2022/10/02/2d-advection/
https://python.keicode.com/lang/regular-expression-finditer.php
"""

'\nhttps://watlab-blog.com/2022/10/08/2d-diffusion/\nhttps://watlab-blog.com/2022/10/02/2d-advection/\nhttps://python.keicode.com/lang/regular-expression-finditer.php\n'

In [2]:
"""
更新箇所

NumPyのベクトル化とNumbaのJITコンパイル
微分方程式の計算と，描画機能を分離


今後


楕円の外の黄色線不要
各seedpointsに対応させて，面積を求める．
この面積はステップごとに分岐をせまられる．
  2倍より大きい　->　分裂    or   大きくする．



細胞分裂の再現 (動的な変化)
時間変化に対して，細胞の大きさが変化する(何をindexとするか?)
細胞の面積を近似計算によって求める．
面積が一定の大きさ(e.x. 2倍)になると分裂させる．

---外側は境界条件，固定されている---

外側も動的に変化させたい．


再考及びtask

微分方程式の離散化について，正当性の検証
離散式の記述を，分かりやすく

単にwus0を前の状態として入れてあるが、よいか？

楕円の境界条件，単にある点が楕円内にあるかどうかで判断

"""

'\n更新箇所\n\n\nNumPyのベクトル化とNumbaのJITコンパイル\n\n\n今後\n\n\n楕円の外の黄色線不要\n各seedpointsに対応させて，面積を求める．\nこの面積はステップごとに分岐をせまられる．\n  2倍より大きい\u3000->\u3000分裂    or   大きくする．\n\n\n\n細胞分裂の再現 (動的な変化)\n時間変化に対して，細胞の大きさが変化する(何をindexとするか?)\n細胞の面積を近似計算によって求める．\n面積が一定の大きさ(e.x. 2倍)になると分裂させる．\n\n---外側は境界条件，固定されている---\n\n外側も動的に変化させたい．\n\n\n再考及びtask\n\n微分方程式の離散化について，正当性の検証\n離散式の記述を，分かりやすく\n\n単にwus0を前の状態として入れてあるが、よいか？\n\n楕円の境界条件，単にある点が楕円内にあるかどうかで判断\n\n'

In [3]:
import numpy as np
from matplotlib import pyplot as plt
import os
import glob
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import re
import pandas as pd
from PIL import Image, ImageDraw
import io
import IPython.display as display
from collections import defaultdict
import sys
from scipy.spatial import Voronoi, voronoi_plot_2d
from sklearn.cluster import KMeans
from numba import jit


from google.colab import files

In [4]:
# csvからparam取得

from google.colab import drive
drive.mount('/content/drive')
Sim_data= pd.read_csv('drive/My Drive/Colab Notebooks/Diffusion-ish/Mesh-type/Sim_Param.csv')


# Parameter_original
abc_idx, abc_par = Sim_data[1:1], Sim_data[0:1]

param_name = []
for n in abc_idx:
    param_name.append(n[0:4])

# pd 1row
param_ori = list(Sim_data.iloc[0])

for a, b in zip(param_name, param_ori):
    a = b

ID, K_CW, n_CW, k_Cr, K_CL, n_CL, K_CH, n_CH, k_Cp, b_Cr, b_Wr, k_WL, k_WC, k_Wr, K_WW, n_WW, K_WL, n_WL, K_WC, n_WC, k_Wp, b_Ww, b_Wr, ki_W, a_Cc, K_HW, n_HW, b_pp, k_pp, b_Hh, k_Hh, k_Ll, b_Ll, D_Ww, D_Cc, D_Ll = [a for a in param_ori]


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
# Initial
wr_sigma11, wr_sigma22, wr_sigma12, wr_sigma21 = 900, 300, 0, 0
wp_sigma11, wp_sigma22, wp_sigma12, wp_sigma21 = 900, 300, 0, 0
cr_sigma11, cr_sigma22, cr_sigma12, cr_sigma21 = 200, 150, 0, 0
cp_sigma11, cp_sigma22, cp_sigma12, cp_sigma21 = 100, 50, 0, 0
eL_sigma11, eL_sigma22, eL_sigma12, eL_sigma21 = 60, 30, 0, 0
eR_sigma11, eR_sigma22, eR_sigma12, eR_sigma21 = 60, 30, 0, 0
h_sigma11, h_sigma22, h_sigma12, h_sigma21 = 10, 10, 0, 0

#            wr    wp    cr    cp    eL    eR     h
sigma11 = [    9,    9,    5,    5,    6,     6,    1]
sigma22 = [    3,    3,  1.5,  1.5,    3,     3,    1]
sigma12 = [    0,    0,    0,    0,    0,     0,    0]
sigma21 = [    0,    0,    0,    0,    0,     0,    0]


# numpyで計算 -> x行目，y行目
mu_wusr_x, mu_wusr_y = 8, 5
mu_wusp_x, mu_wusp_y = 8, 5
mu_clvr_x, mu_clvr_y = 2, 5
mu_clvp_x, mu_clvp_y = 2, 5
mu_erfL_x, mu_erfL_y = 5, 2
mu_erfR_x, mu_erfR_y = 5, 7
mu_ham_x, mu_ham_y = 7, 5

element_str = ["wusr", "wusp", "clvr", "clvp", "erfL", "erfR", "ham"]

# Space
x_size, y_size = 15, 10
x_max, y_max = x_size, y_size  # range
dx, dy = 0.1, 0.1  # divide

# 楕円のパラメータ設定
ellip_long = 7  # 楕円の長軸の半径
ellip_short = 6  # 楕円の短軸の半径
ellip_x = 7.5  # 楕円の中心 x 座標
ellip_y = 2  # 楕円の中心 y 座標

global_seed_points = None

# Amplifier
wr_AMP, wp_AMP, cr_AMP, cp_AMP, eL_AMP, eR_AMP, h_AMP = 600, 150, 300, 150, 300, 300, 10

a = 1   # diffusion rate

# Calculation condition
dt = 0.0005  # Time
step = 20
result_interval = 10


#plsv_wusr, plsv_wusp, plsv_clvr, plsv_clvp, plsv_erfL, plsv_erfR, plsv_ham = 0, 0, 0, 0, 0, 0, 0
plsv_wusr, plsv_wusp, plsv_clvr, plsv_clvp, plsv_erfL, plsv_erfR, plsv_ham = 1, 1, 1, 1, 1, 1, 1

# component
wusr_str, wusp_str, clvr_str, clvp_str, erfL_str, erfR_str, ham_str = 'wusr', 'wusp', 'clvr', 'clvp', 'epflL', 'epflR', 'ham'
c_wusr, c_wusp, c_clvr, c_clvp, c_erfL, c_erfR, c_ham = 'jet', 'seismic', 'twilight_shifted', 'CMRmap', 'cividis', 'plasma', 'terrain'
# , , , , 'gist_stern', 'gnuplot',,, 'brg''gnuplot2' 'CMRmap''cubehelix'

#引数list
pull_sol = [dt, dx, dy, a, step, result_interval]


# GIF用
# 辞書 keylist
keys_name = ["wusr", "wusp", "clvr", "clvp", "erfL", "erfR", "ham"]

img_stock = defaultdict(list)

# 指定されたキーを追加
for key in keys_name:
    img_stock[key]  # defaultdictにキーを生成


In [6]:
def initial_field(x, y):
    # 各成分に対応するゼロ行列を生成
    zero_wusr, zero_wusp, zero_clvr, zero_clvp, zero_erfL, zero_erfR, zero_ham = [np.zeros((len(y), len(x))) for n in range(7)]

    # 2D Gaussian(sigma:分散共分散行列, mu:平均ベクトル)
    wr_sigma, wp_sigma, cr_sigma, cp_sigma, eL_sigma, eR_sigma, h_sigma = [
        np.array([[a, b], [c, d]]) for a, b, c, d in zip(sigma11, sigma12, sigma21, sigma22)
    ]

    # 各成分の行列式と逆行列を計算
    wr_det_sigma, wr_inv_sigma = np.linalg.det(wr_sigma), np.linalg.inv(wr_sigma)
    wp_det_sigma, wp_inv_sigma = np.linalg.det(wp_sigma), np.linalg.inv(wp_sigma)
    cr_det_sigma, cr_inv_sigma = np.linalg.det(cr_sigma), np.linalg.inv(cr_sigma)
    cp_det_sigma, cp_inv_sigma = np.linalg.det(cp_sigma), np.linalg.inv(cp_sigma)
    eL_det_sigma, eL_inv_sigma = np.linalg.det(eL_sigma), np.linalg.inv(eL_sigma)
    eR_det_sigma, eR_inv_sigma = np.linalg.det(eR_sigma), np.linalg.inv(eR_sigma)
    h_det_sigma, h_inv_sigma = np.linalg.det(h_sigma), np.linalg.inv(h_sigma)

    # Gaussian分布を計算する関数
    def get_arr(z, xtemp, ytemp, det_sigma, inv_sigma, AMPP):
        X, Y = np.meshgrid(x, y)
        pos = np.dstack((X, Y))
        arr = np.array([xtemp, ytemp])

        Z = (1 / (2 * np.pi * np.sqrt(det_sigma))) * np.exp(
            -0.5 * np.einsum('...k,kl,...l->...', pos - arr, inv_sigma, pos - arr)
        ) * AMPP
        return Z

    # 各変数のリストを作成
    zero_list = [zero_wusr, zero_wusp, zero_clvr, zero_clvp, zero_erfL, zero_erfR, zero_ham]
    mu_x_list = [mu_wusr_x, mu_wusp_x, mu_clvr_x, mu_clvp_x, mu_erfL_x, mu_erfR_x, mu_ham_x]
    mu_y_list = [mu_wusr_y, mu_wusp_y, mu_clvr_y, mu_clvp_y, mu_erfL_y, mu_erfR_y, mu_ham_y]
    det_list = [wr_det_sigma, wp_det_sigma, cr_det_sigma, cp_det_sigma, eL_det_sigma, eR_det_sigma, h_det_sigma]
    inv_list = [wr_inv_sigma, wp_inv_sigma, cr_inv_sigma, cp_inv_sigma, eL_inv_sigma, eR_inv_sigma, h_inv_sigma]
    AMP_list = [wr_AMP, wp_AMP, cr_AMP, cp_AMP, eL_AMP, eR_AMP, h_AMP]

    # 各成分のGaussian分布を計算
    zzz_wusr, zzz_wusp, zzz_clvr, zzz_clvp, zzz_erfL, zzz_erfR, zzz_ham = [
        get_arr(a, b, c, d, e, f) for a, b, c, d, e, f in zip(zero_list, mu_x_list, mu_y_list, det_list, inv_list, AMP_list)
    ]

    # 計算結果を返す
    return zzz_wusr, zzz_wusp, zzz_clvr, zzz_clvp, zzz_erfL, zzz_erfR, zzz_ham



def boundary_condition(a):
    a[:, 0], a[:, -1], a[0, :], a[-1, :] = 0, 0, 0, 0  # left, right, up. bottom
    return a


In [7]:
def seed_points_maker():
    """楕円内部にシードポイントを生成してグローバル変数に格納"""
    global global_seed_points

    # シードポイントの生成
    np.random.seed(100)
    points = []
    while len(points) < 80:
        px = np.random.uniform(ellip_x - ellip_long, ellip_x + ellip_long)
        py = np.random.uniform(ellip_y - ellip_short, ellip_y + ellip_short)
        if ((px - ellip_x) ** 2 / ellip_long ** 2 + (py - ellip_y) ** 2 / ellip_short ** 2) <= 1:
            points.append([px, py])

    global_seed_points = np.array(points)

In [8]:
@jit(nopython=True)
def update_values(wusr, wusp, clvr, clvp, erfL, erfR, ham, wusr0, wusp0, clvr0, clvp0, erfL0, erfR0, ham0,
                  dt, dx, dy, masks, k_Wr, k_WL, K_WL, n_WL, k_WC, K_WC, n_WC, ki_W, K_WW, n_WW, b_Wr,
                  k_Wp, b_Ww, D_Ww, k_Cr, K_CL, n_CL, a_Cc, K_CW, n_CW, K_CH, n_CH, b_Cr, k_Cp, D_Cc,
                  k_Ll, b_Ll, D_Ll, k_pp, b_pp):
    """要素ごとの更新"""
    for i in range(1, wusr.shape[0] - 1):
        for j in range(1, wusr.shape[1] - 1):
            if np.any(masks[:, j, i] == 1):  # ボロノイ領域内の点のみ
                wusr[i, j] = wusr0[i, j] + dt * (
                    k_Wr * ((k_WL / (1 + ((erfL0[i, j] + erfR0[i, j]) / K_WL) ** n_WL)) +
                            (k_WC / (1 + ((clvp0[i, j]) / K_WC) ** n_WC))) *
                    (ki_W + ((wusp0[i, j] / K_WW) ** n_WW / (1 + (wusp0[i, j] / K_WW) ** n_WW))) - b_Wr * wusr0[i, j])

                wusp[i, j] = wusp0[i, j] + dt * (
                    k_Wp * wusr0[i, j] - b_Ww * wusp0[i, j] +
                    D_Ww * ((wusp0[i + 1, j] - 2 * wusp0[i, j] + wusp0[i - 1, j]) / dx ** 2) +
                    D_Ww * ((wusp0[i, j + 1] - 2 * wusp0[i, j] + wusp0[i, j - 1]) / dy ** 2))

                clvr[i, j] = clvr0[i, j] + dt * (
                    (k_Cr / (1 + ((erfL0[i, j] + erfR0[i, j]) / K_CL) ** n_CL)) *
                    (a_Cc + ((wusp0[i, j] / K_CW) ** n_CW / (1 + (wusp0[i, j] / K_CW) ** n_CW))) *
                    (1 / (1 + (ham0[i, j] / K_CH) ** n_CH)) - b_Cr * clvr0[i, j])

                clvp[i, j] = clvp0[i, j] + dt * (
                    k_Cp * clvr0[i, j] - b_Cr * clvp0[i, j] +
                    D_Cc * ((clvp0[i + 1, j] - 2 * clvp0[i, j] + clvp0[i - 1, j]) / dx ** 2) +
                    D_Cc * ((clvp0[i, j + 1] - 2 * clvp0[i, j] + clvp0[i, j - 1]) / dy ** 2))

                erfL[i, j] = erfL0[i, j] + dt * (
                    k_Ll - b_Ll * erfL0[i, j] +
                    D_Ll * ((erfL0[i + 1, j] - 2 * erfL0[i, j] + erfL0[i - 1, j]) / dx ** 2) +
                    D_Ll * ((erfL0[i, j + 1] - 2 * erfL0[i, j] + erfL0[i, j - 1]) / dy ** 2))

                erfR[i, j] = erfR0[i, j] + dt * (
                    k_Ll - b_Ll * erfR0[i, j] +
                    D_Ll * ((erfR0[i + 1, j] - 2 * erfR0[i, j] + erfR0[i - 1, j]) / dx ** 2) +
                    D_Ll * ((erfR0[i, j + 1] - 2 * erfR0[i, j] + erfR0[i, j - 1]) / dy ** 2))

                ham[i, j] = ham0[i, j] + dt * (k_pp - b_pp * ham0[i, j])
            else:
                # 楕円の外部の要素は0にする
                wusr[i, j] = 0
                wusp[i, j] = 0
                clvr[i, j] = 0
                clvp[i, j] = 0
                erfL[i, j] = 0
                erfR[i, j] = 0
                ham[i, j] = 0


def sol_diffusion(x, y, wusr, wusp, clvr, clvp, erfL, erfR, ham, pull, ID):
    """2次元拡散方程式を計算し、結果を返す"""
    dt, dx, dy, a, step, result_interval = pull
    wusr, wusp, clvr, clvp, erfL, erfR, ham = wusr.T, wusp.T, clvr.T, clvp.T, erfL.T, erfR.T, ham.T

    # グローバル変数からシードポイントを取得
    global global_seed_points
    points = global_seed_points

    # Voronoi
    vor = Voronoi(points)

    # 各点に対応するマスクを作成
    masks = np.zeros((len(points), len(y), len(x)))
    for i, (px, py) in enumerate(points):
        for j in range(len(x)):
            for k in range(len(y)):
                point_index = vor.point_region[i]
                vertices = vor.regions[point_index]
                if all(v >= 0 for v in vertices):
                    vert = np.array([vor.vertices[v] for v in vertices])
                    if np.min(vert[:, 0]) <= x[j] <= np.max(vert[:, 0]) and np.min(vert[:, 1]) <= y[k] <= np.max(vert[:, 1]):
                        masks[i, k, j] = 1


    results = {
        'wusr': [],
        'wusp': [],
        'clvr': [],
        'clvp': [],
        'erfL': [],
        'erfR': [],
        'ham': [],
        'n': []
    }


    for n in range(step):
        wusr0, wusp0, clvr0, clvp0, erfL0, erfR0, ham0 = wusr.copy(), wusp.copy(), clvr.copy(), clvp.copy(), erfL.copy(), erfR.copy(), ham.copy()
        update_values(wusr, wusp, clvr, clvp, erfL, erfR, ham, wusr0, wusp0, clvr0, clvp0, erfL0, erfR0, ham0,
                      dt, dx, dy, masks, k_Wr, k_WL, K_WL, n_WL, k_WC, K_WC, n_WC, ki_W, K_WW, n_WW, b_Wr,
                      k_Wp, b_Ww, D_Ww, k_Cr, K_CL, n_CL, a_Cc, K_CW, n_CW, K_CH, n_CH, b_Cr, k_Cp, D_Cc,
                      k_Ll, b_Ll, D_Ll, k_pp, b_pp)
        if n % result_interval == 0:
            # 結果を保存
            results['wusr'].append(wusr.copy())
            results['wusp'].append(wusp.copy())
            results['clvr'].append(clvr.copy())
            results['clvp'].append(clvp.copy())
            results['erfL'].append(erfL.copy())
            results['erfR'].append(erfR.copy())
            results['ham'].append(ham.copy())
            results['n'].append(n)

    return results, points, vor



In [9]:
def plot_voronoi(vor, points, ellip_x, ellip_y, ellip_long, ellip_short):
    """楕円内でのボロノイ分割の描画"""
    fig, ax = plt.subplots(figsize=(8, 8))
    voronoi_plot_2d(vor, ax=ax, show_vertices=False, line_colors='yellow',  # ボロノイ境界を黄色に
                    line_width=2, line_alpha=0.6, point_size=10)

    # シードポイントの描画
    ax.plot(points[:, 0], points[:, 1], 'bo', markersize=8)

    # 楕円の描画
    theta = np.linspace(0, 2 * np.pi, 100)
    ellipse_x = ellip_x + ellip_long * np.cos(theta)
    ellipse_y = ellip_y + ellip_short * np.sin(theta)
    ax.plot(ellipse_x, ellipse_y, 'w--', linewidth=2)  # 楕円の境界を白に

    # x軸とy軸の範囲を設定
    ax.set_xlim(0, x_size)
    ax.set_ylim(0, y_size)
    plt.title('Voronoi Diagram within Ellipse')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.grid(True)
    plt.show()


def plot(x, y, z, n, name, color, ID, vor=None, points=None, ellip_x=None, ellip_y=None, ellip_long=None, ellip_short=None):
    plt.rcParams['font.size'] = 14
    plt.rcParams['xtick.direction'] = 'in'
    plt.rcParams['ytick.direction'] = 'in'

    # x_size = 8
    # y_size = int(0.8 * x_size * (np.max(y) / np.max(x)))
    fig, ax1 = plt.subplots(figsize=(x_size, y_size))
    ax1.yaxis.set_ticks_position('both')
    ax1.xaxis.set_ticks_position('both')
    ax1.set_xlabel('x_label')
    ax1.set_ylabel('y_label')

    # メインのデータの描画
    im = ax1.imshow(z, vmin=0, vmax=100, extent=[0, x_size, 0, y_size], aspect='equal', cmap=f'{color}')
    ax1.text(0.1, 0.1, f'{name}, Step={n}', color="black")
    ax1.text(7, 0.1, f'{name}, Step={n}', color="white")
    cbar = fig.colorbar(im)
    cbar.set_label(f'{name}')

    # ボロノイ分割とシードポイントの描画
    if vor is not None and points is not None:
        voronoi_plot_2d(vor, ax=ax1, show_vertices=False, line_colors='yellow', line_width=2, line_alpha=0.6, point_size=10)
        ax1.plot(points[:, 0], points[:, 1], 'bo', markersize=8)
        theta = np.linspace(0, 2 * np.pi, 100)
        ellipse_x = ellip_x + ellip_long * np.cos(theta)
        ellipse_y = ellip_y + ellip_short * np.sin(theta)
        ax1.plot(ellipse_x, ellipse_y, 'w--', linewidth=2)  # 楕円の境界を白に

    # x軸とy軸の範囲を設定
    ax1.set_xlim(0, x_size)
    ax1.set_ylim(0, y_size)

    # メモリ上に保存
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)
    img_stock[name].append(Image.open(buf))

    plt.show()
    plt.close(fig)


In [21]:
def plot_diffusion(x, y, element_var, points, vor, ellip_x, ellip_y, ellip_long, ellip_short, ID):
    n_list = results['n']
    for idx, n in enumerate(n_list):
        element_var = [
            results['wusr'][idx].T,
            results['wusp'][idx].T,
            results['clvr'][idx].T,
            results['clvp'][idx].T,
            results['erfL'][idx].T,
            results['erfR'][idx].T,
            results['ham'][idx].T
        ]
        for field, name, color in zip(
            element_var,
            ["wusr", "wusp", "clvr", "clvp", "erfL", "erfR", "ham"],
            [c_wusr, c_wusp, c_clvr, c_clvp, c_erfL, c_erfR, c_ham]
        ):
            if globals().get(f'plsv_{name}', True):
                plot(
                    x, y, field, n=n, name=name, color=color, ID=ID, vor=vor, points=points,
                    ellip_x=ellip_x, ellip_y=ellip_y,
                    ellip_long=ellip_long, ellip_short=ellip_short
                )

In [17]:

# Initial field
x, y = np.linspace(0, x_max, int(x_max / dx)), np.linspace(0, y_max, int(y_max / dy))
wusr, wusp, clvr, clvp, erfL, erfR, ham = initial_field(x, y)


# 境界条件を設定する
wusr = boundary_condition(wusr)
wusp = boundary_condition(wusp)
clvr = boundary_condition(clvr)
clvp = boundary_condition(clvp)
erfL = boundary_condition(erfL)
erfR = boundary_condition(erfR)
ham = boundary_condition(ham)


# 安定性の確認
nu_x, nu_y = a * dt / dx ** 2, a * dt / dy ** 2
print('nu_x, nu_y=', nu_x, nu_y)

seed_points_maker()


pull_sol = [dt, dx, dy, a, step, result_interval]
ID = 0
results, points, vor = sol_diffusion(x, y, wusr, wusp, clvr, clvp, erfL, erfR, ham, pull_sol, ID)


nu_x, nu_y= 0.04999999999999999 0.04999999999999999


In [22]:

plot_diffusion(x, y, results, points, vor, ellip_x, ellip_y, ellip_long, ellip_short, ID)


Output hidden; open in https://colab.research.google.com to view.

In [ ]:
  """
  # GIF作成
  for z_key, images in img_stock.items():
      # 一時的に画像を保存するリストを作成
      temp_images = []

      for idx, image in enumerate(images):
          # 各フレームをボロノイ情報を含む画像として保存する
          fig, ax = plt.subplots()
          # 画像としてプロットされたシミュレーション結果を表示
          ax.imshow(np.asarray(image), vmin=0, vmax=100, aspect='equal', cmap=c_wusr)

          # ボロノイ情報の重ね合わせ
          if vor is not None and points is not None:
              voronoi_plot_2d(vor, ax=ax, show_vertices=False, line_colors='yellow', line_width=2, line_alpha=0.6, point_size=10)
              ax.plot(points[:, 0], points[:, 1], 'bo', markersize=8)
              theta = np.linspace(0, 2 * np.pi, 100)
              ellipse_x = ellip_x + ellip_long * np.cos(theta)
              ellipse_y = ellip_y + ellip_short * np.sin(theta)
              ax.plot(ellipse_x, ellipse_y, 'w--', linewidth=2)  # 楕円の境界を白に

          # バッファに保存
          buf = io.BytesIO()
          plt.savefig(buf, format='png')
          buf.seek(0)
          temp_images.append(Image.open(buf))
          plt.close(fig)

      # 保存された一連の画像からGIFを作成
      gif_bytes = io.BytesIO()
      temp_images[0].save(gif_bytes, format='GIF', save_all=True, append_images=temp_images[1:], duration=900, loop=0)

      # GIFを表示
      gif_bytes.seek(0)
      display.display(display.Image(data=gif_bytes.getvalue(), format='png'))

      gif_filename = f"{z_key}_animation.gif"
  """